In [ ]:
from pyspark.sql import functions as F

# Caminho de origem dos arquivos JSON brutos no Volume
SOURCE_PATH = "/Volumes/spotify_analytics/landing/raw_data/spotify_data_raw_*.json"

# Configuração destino
CATALOGO_DESTINO = "spotify_analytics"
SCHEMA_DESTINO = "bronze"
TABELA_DESTINO = "tb_bronze_search"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### Configuração

Define:
- **SOURCE_PATH**: Localização dos arquivos JSON no Unity Catalog Volume (landing layer)
- **Destino**: Tabela Bronze onde serão armazenados os dados brutos

### Leitura de Arquivos JSON como Texto

Estratégia: Lê JSON completo como STRING para preservar estrutura original.
- Usa `format("text")` com `wholetext=true` para capturar arquivo inteiro
- Adiciona metadata do arquivo fonte usando `_metadata.file_path` (Unity Catalog)
- Adiciona timestamp de ingestão

In [ ]:
# Lê arquivos JSON como texto bruto
df_raw = (
    spark.read
    .format("text")
    .option("wholetext", "true")
    .load(SOURCE_PATH)
)

# Adiciona metadata (Unity Catalog usa _metadata.file_path)
df_bronze = (
    df_raw
    .withColumnRenamed("value", "raw_json")
    .withColumn("source_file", F.col("_metadata.file_path"))
    .withColumn("ingestion_date", F.current_timestamp())
)

print(f"Total de arquivos lidos: {df_bronze.count()}")

### Carga na Tabela Bronze

Escreve dados na tabela Delta usando modo **APPEND**:
- Permite cargas incrementais
- Mantém histórico de todas as ingestões
- `mergeSchema=true` para flexibilidade de schema

In [ ]:
(
    df_bronze.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(nome_tabela_destino)
)

print(f"✅ Dados carregados com sucesso em {nome_tabela_destino}")

### Verificação Final

Confirma quantidade de registros na tabela Bronze.

In [ ]:
total_registros = spark.table(nome_tabela_destino).count()
print(f"Total de registros em {nome_tabela_destino}: {total_registros}")

# Mostra últimas ingestões
spark.sql(f"""
    SELECT ingestion_date, COUNT(*) as qtd_registros
    FROM {nome_tabela_destino}
    GROUP BY ingestion_date
    ORDER BY ingestion_date DESC
    LIMIT 5
""").show()